# Vorbereitung

In [1]:
%env KERAS_BACKEND=theano
%env THEANO_FLAGS=floatX=float32,device=cpu

env: KERAS_BACKEND=theano
env: THEANO_FLAGS=floatX=float32,device=cpu


In [2]:
%load_ext autoreload
%autoreload 1
%aimport bb_behavior
%aimport bb_behavior.plot
%aimport bb_behavior.tracking
%aimport bb_behavior.tracking.pipeline

import bb_behavior
import bb_behavior.plot
import bb_behavior.tracking
import bb_behavior.tracking.pipeline

import pandas as pd
import time
import datetime

Using Theano backend.


# Preprocessing

## Preprocessing step 1: convert the h264 to mp4

For examining videos

In [3]:
import glob
import os
import subprocess

videos = "../videos/videos_tags/videos_tags_mp4"

if not os.path.exists(videos):
    os.mkdir(videos)

In [5]:
for video in glob.glob(os.path.join('../videos/videos_tags/', '*.h264')):  
    path = os.path.split(video)   
    video_mp4 = os.path.join(path[0],"videos_tags_mp4",os.path.splitext(path[1])[0] + '.mp4')

    print(video_mp4)
    subprocess.call(['ffmpeg', '-framerate', '10', '-fflags', '+genpts', '-i', video, '-c', 'copy', video_mp4])

../videos/videos_tags/videos_tags_mp4/e00_2018-08-19-01-08-13.mp4
../videos/videos_tags/videos_tags_mp4/e00_2018-08-19-01-08-33.mp4
../videos/videos_tags/videos_tags_mp4/e00_2018-08-19-01-08-54.mp4
../videos/videos_tags/videos_tags_mp4/e00_2018-08-19-02-20-55.mp4
../videos/videos_tags/videos_tags_mp4/e00_2018-08-19-02-21-16.mp4
../videos/videos_tags/videos_tags_mp4/e00_2018-08-19-02-21-37.mp4
../videos/videos_tags/videos_tags_mp4/e00_2018-08-19-02-21-58.mp4
../videos/videos_tags/videos_tags_mp4/e00_2018-08-19-02-22-19.mp4
../videos/videos_tags/videos_tags_mp4/e00_2018-08-19-02-22-40.mp4
../videos/videos_tags/videos_tags_mp4/e00_2018-08-19-02-23-01.mp4
../videos/videos_tags/videos_tags_mp4/e00_2018-08-19-02-23-22.mp4
../videos/videos_tags/videos_tags_mp4/e00_2018-08-19-02-23-43.mp4
../videos/videos_tags/videos_tags_mp4/e00_2018-08-19-02-24-04.mp4
../videos/videos_tags/videos_tags_mp4/e00_2018-08-19-02-24-25.mp4
../videos/videos_tags/videos_tags_mp4/e00_2018-08-19-02-25-33.mp4
../videos/

KeyboardInterrupt: 

## Run

In [4]:
from bb_behavior.tracking.pipeline import get_default_pipeline
default_pipeline = None
default_pipeline = get_default_pipeline(localizer_threshold="0.50")

/home/tscho/.local/lib/python3.6/site-packages/keras/backend/theano_backend.py:1032: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')
/home/tscho/.local/lib/python3.6/site-packages/keras/backend/theano_backend.py:1032: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/home/tscho/.local/lib/python3.6/site-packages/keras/backend/theano_backend.py:1032: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='max')


In [5]:
from tqdm import tqdm_notebook # progress bar

import math
import numpy as np
from bb_tracking.data.constants import DETKEY
#from bb_tracking.tracking import score_id_sim_v
from bb_tracking.tracking import distance_orientations_v, distance_positions_v

from bb_behavior.tracking.pipeline import detect_markers_in_video
from bb_behavior.tracking.pipeline import track_detections_dataframe
from bb_behavior.tracking.pipeline import display_tracking_results

In [6]:
config = dict(tag_pixel_diameter=50,
              n_frames=None,
              confidence_filter_detections=0.08,
              confidence_filter_tracks=0.20,
              coordinate_scale=1.0,
              start_time=None,
              fps=10.0,
              cam_id=0,
              left_leaving_area = 0.30, # Prozente vom Bildschirmrand, zB. bei 1000px und 0.15 -> 0-150px
              right_leaving_area = 0.30,
              px_x_resolution_vid = 1944)

In [9]:
import time
import os

def filename_to_datestring(filname):
    """
    filename can be path
    """
    return os.path.split(filname)[-1].split('.')[0].split('_')[-1]


def string_to_timestamp(datestring):
    """ 
    params
        string: format 2018-08-19-01-08-13
    output
        unix timestamp (float)
    """
    
    return time.mktime(time.strptime(datestring, "%Y-%m-%d-%H-%M-%S"))

def timestamp_to_string(timestamp):
    return time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime(timestamp))

# Get all videos from test set

In [10]:
# read in the test data csv
test_data = pd.read_csv('bees_test.csv')

# convert the full filenames to string timestamps, and sort by timestamp_in
test_data['video'] = test_data['video'].apply(lambda x: filename_to_datestring(x))
test_data['timestamp_in'] = test_data['timestamp_in'].apply(lambda x: filename_to_datestring(x))
test_data['timestamp_out'] = test_data['timestamp_out'].apply(lambda x: filename_to_datestring(x))
test_data.sort_values(['timestamp_in'])

test_data.head()

,bee_id,in_direction,out_direction,video,timestamp_in,timestamp_out
0,855,right,left,2018-08-19-05-25-51,2018-08-19-05-25-51,2018-08-19-05-25-51
1,220,left,right,2018-08-19-05-25-51,2018-08-19-05-25-51,2018-08-19-05-25-51
2,1301,left,left,2018-08-19-18-33-03,2018-08-19-18-32-42,2018-08-19-18-33-03
3,951,left,left,2018-08-19-18-33-03,2018-08-19-18-33-03,2018-08-19-18-33-03
4,1520,left,left,2018-08-19-18-33-03,2018-08-19-18-33-03,2018-08-19-18-33-03


In [11]:
# get all videos between timestamp_in and timestamp_out

all_paths = pd.DataFrame(glob.glob(os.path.join('../videos/videos_tags/', '*.h264')), columns=['video'])

all_paths['video'] = all_paths['video'].apply(lambda x: filename_to_datestring(x))
all_paths.sort_values(['video'])

def get_videos_between(timestamp_in, timestamp_out):
    """ returns all video between timestamp_in and timestamp out (inclusive) 
    timestamps are string timestamps
    """
    
    mask = (all_paths['video'] >= timestamp_in) & (all_paths['video'] <= timestamp_out)
    return list(all_paths[mask]['video'])

all_paths.head()

,video
0,2018-08-19-01-08-13
1,2018-08-19-01-08-33
2,2018-08-19-01-08-54
3,2018-08-19-02-20-55
4,2018-08-19-02-21-16


In [12]:
all_videos = []

# go through test_data and get all videos between timestamp_in and timestamp_out
for index, row in test_data.iterrows():
    
    vids = get_videos_between(row['video_start_time'],row['video_end_time'])
    if len(vids) < 6:
        all_videos += vids
    
all_videos = sorted(list(set(all_videos)))
all_videos

KeyError: 'video_start_time'

# Tracking

In [ ]:
# Einen Iterable speichern, der alle Videos in einem Iterable zur Verfügung stellt
# Diesen Iterable können wir dann in der nächsten Zelle mit tqdm schön durchlaufen
# nicht sehr performant --> O(n)
base_directory = "../videos/videos_tags/"

paths = []
for video_name in all_videos:
    paths.append(base_directory + "e00_" + video_name + ".h264")

In [ ]:
paths

In [ ]:
# Hier passiert das eigentliche Tracken und speichern der Ergebnisse:
num_processed_videos = 0
video_data = dict()
frame_info = None
detections = None
videos_w_no_detection = ""
import glob

#paths = glob.glob(os.path.join('../videos/videos_tags/', '*.h264'))[:5]
#paths = ['../videos/videos_tags/e00_2018-08-19-02-20-55.h264']

bad_paths = []

#paths = ['../videos/videos_tags/e00_2018-08-19-12-34-35.h264']

for path in tqdm_notebook(paths):
    start_time = config["start_time"]
    cam_id = config["cam_id"]
    try:
        
        num_processed_videos += 1
        
        print(num_processed_videos)

        frame_info, detections = detect_markers_in_video(path,
                                                      decoder_pipeline=default_pipeline,#pipeline=pipelines(),
                                                     tag_pixel_diameter=config["tag_pixel_diameter"],
                                                      n_frames=config["n_frames"],
                                                      fps=config["fps"],
                                                     progress="tqdm_notebook",
                                                         verbose = False
                                            )
        if len(detections[detections['confidence']>=config["confidence_filter_detections"]]) == 0:
            continue
            
        tracks = track_detections_dataframe(detections,
                                            tracker="../tracker.det_score_fun.frag_score_fun.dill",
                                            confidence_filter_detections=config["confidence_filter_detections"],
                                           confidence_filter_tracks=config["confidence_filter_tracks"],
                                            coordinate_scale=config["coordinate_scale"]
                                            
                                           )
        date_string = filename_to_datestring(path)
        tracks['video'] = date_string
        video_data[path] = (frame_info, detections, tracks)
    except ValueError as err: #tritt auf, wenn Video leer ist. In diesem Fall: überspringe video
        try:
            videos_w_no_detection = videos_w_no_detection + path + "\n"
            bad_paths.append(path)
            # wir arbeiten später nochmal mit paths, daher müssen das leere löschen, weil sonst
            # in video_data kein zugehöriger Value zu Key = file zu finden ist.
        except KeyError as err:
            print(err)
            #continue
            raise
    except Exception as err:
        print(err)
        raise
    # only first vid: break

In [ ]:
for path in bad_paths:
    paths.remove(path)

In [ ]:
# gebe Videos aus, in denen keine Bienen gefunden wurden
if videos_w_no_detection:
    print("In the following file(s) were no bees detected:")
    print(videos_w_no_detection)

In [ ]:
paths

In [ ]:
#video_data["file"][0] --> frame-info, [1] --> detections, [2] --> tracks
for path in paths:
    display_tracking_results(path, video_data[path][0], video_data[path][1], video_data[path][2])

In [ ]:
tracks = [video_data[paths[x]][2] for x in range(len(paths))]
tracks = pd.concat(tracks,ignore_index=True)
tracks = tracks.drop(columns=["localizerSaliency", "beeID", "camID", "frameIdx"])

In [ ]:
with open("all_tracks.csv", "w") as f:
    tracks.to_csv(f)

In [ ]:
tracks

# Algorithmus 1: Baseline

In [ ]:
# ERGEBNIS-DICT, kann mit pd.DataFrame(data=bee_moves) in DataFrame gewandelt werden
bee_moves = {"bee_id":[], "time_in":[], "os_in":[], "time_out":[], "os_out":[], "in_direction":[], "out_direction":[]}

from math import pi
""" Vorgehen:
Für jedes Video
- Prüfe, wo sich die Biene beim ersten erkannten Erscheinen aufhält
    - Ordne die Position in left, right oder middle ein
- Prüfe, wo sich die Biene beim letzten erkannten Erscheinen aufhält
    - Ordne die Position in left, right oder middle ein
Wenn sich Biene in der Mitte befindet, starte neue Routine, die links oder rechts zuordnet
Daraus kann nun abgeleitet werden, wo die Biene reingekommen ist und wo sie rausgegangen ist.
"""

# Routine, wenn Biene in der Mitte
# Betrachte zRotation, das ist die Richtung in die die Biene guckt (in Bogenmaß)
# gebe diese Richtung aus
def middle_classifier(zRotation):
    if zRotation > abs(pi/2):
        return "left"
    else:
        return "right"

def baseline_alg_classify_bee():
    video_groups = tracks.groupby(["video"])
    for group_name, df_group in video_groups:
        movement = dict()
        recording_begin = group_name

        # Finde start- und endposition
        for row in df_group.itertuples():
            bee_id = row[9]
            xpos = row[2]
            ypos = row[3]
            zRotation = row[4]
            timestamp = row[5]

            if bee_id in movement.keys():
                if not movement[bee_id]: #hier wird value betrachtet
                    # Aufbau movement[bee_id] = 
                    # [(xpos_anfang, ypos_anfang), (xpos_ende, ypos_ende), time_in bzw. time_out, os_in, os_out, z_start, z_end]
                    movement[bee_id] = [(xpos, ypos), (xpos, ypos), recording_begin, timestamp, timestamp, zRotation, zRotation]
                else: #spätere detection in Video -> aktualisiere
                    movement[bee_id][1] = (xpos, ypos)
                    movement[bee_id][4] = timestamp
                    movement[bee_id][6] = zRotation
            else:
                movement[bee_id] = [(xpos, ypos), (xpos, ypos), recording_begin, timestamp, timestamp, zRotation, zRotation]

        def is_left(xpos):
            return xpos <= config["left_leaving_area"]*config["px_x_resolution_vid"]
        def is_right(xpos):
            return xpos >= config["px_x_resolution_vid"] - config["right_leaving_area"]*config["px_x_resolution_vid"]

        # Setze labels, entferne andere Daten
        for bee_id in movement.keys():
            start_pos = movement[bee_id][0]
            end_pos = movement[bee_id][1]
            z_start = movement[bee_id][-2]
            z_end = movement[bee_id][-1]
            labels = dict()

            # Eintrittsseite festlegen
            if is_left(start_pos[0]):
                labels["in_direction"] = "left"
            elif is_right(start_pos[0]):
                labels["in_direction"] = "right"
            else:
                labels["in_direction"] = middle_classifier(z_start)

            # Austrittsseite festlegen
            if is_left(end_pos[0]):
                labels["out_direction"] = "left"
            elif is_right(end_pos[0]):
                labels["out_direction"] = "right"
            else:
                labels["out_direction"] = middle_classifier(z_end)

            bee_moves["bee_id"].append(bee_id)
            bee_moves["in_direction"].append(labels["in_direction"])
            bee_moves["out_direction"].append(labels["out_direction"])
            bee_moves["time_in"].append(movement[bee_id][2])
            bee_moves["time_out"].append(movement[bee_id][2])
            bee_moves["os_in"].append(movement[bee_id][3])
            bee_moves["os_out"].append(movement[bee_id][4])
            
            # write to MaschineLearnedData.csv
            attache = 'a'
            if not (os.path.isfile('./MaschineLearnedData.csv')):
                attache = 'w'
                with open('MaschineLearnedData.csv', attache) as csvfile:
                    filewriter = csv.writer(csvfile, delimiter=',', quotechar='\"', quoting=csv.QUOTE_MINIMAL)
                    filewriter.writerow(["bee_id", "time_in", "os_in", "time_out", "os_out", "in_direction", "out_direction"])
            else:
                with open('MaschineLearnedData.csv', attache) as csvfile:
                    filewriter = csv.writer(csvfile, delimiter=',', quotechar='\'', quoting=csv.QUOTE_MINIMAL)
                    for i in range(0,len(bee_moves["bee_id"])):
                        filewriter.writerow([bee_moves["bee_id"][i], bee_moves["time_in"][i], bee_moves["os_in"][i], bee_moves["time_out"][i], bee_moves["os_out"][i], bee_moves["in_direction"][i], bee_moves["out_direction"][i]])

In [ ]:
# Ausgeben
baseline_alg_classify_bee()
bee_moves = pd.DataFrame(data=bee_moves)
bee_moves

# Compare-Algorithm

In [ ]:
def formatFileName(filename):
    return filename.split('_')[-1].split('.')[0]

In [ ]:
""" 
Precondition:
Fetch computed data and GroundTruthData from generated csv files and store in dictionaries.
"""

# read data from GroundTruthData.csv files
import csv
# use Dictionary to store data we want to compare
dicManually = {} # key - List
dicAutomated = {} # key - List

# fetching data from MaschineLearnedData.csv file 
with open("MaschineLearnedData.csv", "r") as testFile:
    """ fetch all data from csv file to store in dicAutomated for better search and use """
    reader = csv.DictReader(testFile)
    for row in reader:
        # store video in dicAutomated
        if not (row['time_in'] in dicAutomated):
            # add video to dic with his bees
            dicAutomated[row['time_in']] = [[str(int(float(row['bee_id']))), row['in_direction'], row['out_direction']]]
        else:
            #add other bees to the video key
            dicAutomated[row['time_in']].append([str(int(float(row['bee_id']))), row['in_direction'], row['out_direction']])

# fetching data from GroundTruthData.csv
with open("GroundTruthData.csv", "r") as testFile:
    """ fetch all data from csv file to store in diManually for better search and use """
    reader = csv.DictReader(testFile)
    for row in reader:
        # check row has an bee_id -> if nor go to next row
        if (row['bee_id'] != ""):
            # store video in dicManually 
            if not (formatFileName(row['video']) in dicManually):
                # add video to dic with his bees
                dicManually[formatFileName(row['video'])] = [[row['bee_id'], row['in_direction'], row['out_direction']]]
            else:
                #add other bees to the video key
                dicManually[formatFileName(row['video'])].append([row['bee_id'], row['in_direction'], row['out_direction']])
        else:
            continue

In [ ]:
""" Compare input of MaschineLearnedData.csv with GroundTruthData.csv to get just compareable tracks """

# delete values which are not included in dicAutomated
for msData in list(dicManually):
    if not (msData in dicAutomated):
        dicManually.pop(msData)
        
# delete values which are noch included in dicManually
for msData in list(dicAutomated):
    if not (msData in dicManually):
        dicAutomated.pop(msData)

In [ ]:
import os

def writeToCSV(beeid, video, indirection, outdirection):
    # write to MaschineLearnedData.csv
            attache = 'a'
            if not (os.path.isfile('./result.csv')):
                attache = 'w'
                with open('result.csv', attache) as csvfile:
                    filewriter = csv.writer(csvfile, delimiter=',', quotechar='\"', quoting=csv.QUOTE_MINIMAL)
                    filewriter.writerow(["bee_id", "video", "in_direction", "out_directiomn"])
                    
            with open('result.csv', attache) as csvfile:
                filewriter = csv.writer(csvfile, delimiter=',', quotechar='\'', quoting=csv.QUOTE_MINIMAL)
                filewriter.writerow([beeid, video, indirection, outdirection])

In [ ]:
""" compare data of both dictionaries """
# take key of dicAutomated as long as it has a value -> if empty list: delete key from dic
# kompare key value with key value of dicManually + delete compared key-value of both dic's

_correctMatches = 0
_allMatches = 0

for bee in list(dicAutomated):
    while dicAutomated[bee] != []:       
        # find data in dicManually and compare data before delete in dicManually
        _allMatches += 1
        currAutVal = dicAutomated[bee][0]
        currManVal = []
        
        # loop through dicManually Value to find bee_id
        inside = False
        for beeid in list(dicManually[bee]):
            if (beeid[0] == currAutVal[0]):
                inside = True
                currManVal = beeid
                # start easy compare
                if (currAutVal[1] == currManVal[1] and currAutVal[2] == currManVal[2]):
                    _correctMatches += 1
                    writeToCSV(beeid[0], bee, "", "")
                elif (currAutVal[1] != currManVal[1] and currAutVal[2] != currManVal[2]):
                    _correctMatches += 0
                    writeToCSV(beeid[0], bee, "x", "x")
                elif (currAutVal[1] == currManVal[1] and currAutVal[2] != currManVal[2]):
                    _correctMatches += 0.5
                    writeToCSV(beeid[0], bee, "", "x")
                else:
                    _correctMatches += 0.5
                    writeToCSV(beeid[0], bee, "x", "")
                
        if (inside == False):    
            print('Error: No matching bee found!')
                
        #print(currManVal, " \n " , currAutVal, "\n\n") # comparable data
        del dicAutomated[bee][0]
    dicAutomated.pop(bee)
    
try:
    print(round(_correctMatches / _allMatches, 4) * 100, "% correctness")
except ZeroDivisionError:
    print("Nothing to compare!")

# Vergleichsfunktion Algorithmus mit Labels

In [ ]:
# gelabelted Datensatz einlesen:
labeled_data = pd.read_csv("labels.csv")
labeled_data = labeled_data.filter(["bee_id", "video", "in_direction", "out_direction"])

# nur ein Video betrachten:
#labeled_data = labeled_data[labeled_data["video"] == "e00_2018-08-19-12-34-35.h264"].drop(columns=["video"])

# Vergleichen

for bee_id in results["bee_id"]:
    if results[bee_id] in labeled_data["bee_id"]:
        if results[bee_id]
    

# Zwischenschritt für ML-Algorithmus

In [13]:
tracks = pd.read_csv('all_tracks.csv', index_col=0)

In [14]:
tracks.head()

,id,xpos,ypos,zrotation,timestamp,confidence,track_id,track_confidence,bee_id,video
0,f16454994143232751227d0c0,477.180838,164.151279,0.270257,8.1,0.183379,0.0,0.831785,433.0,2018-08-19-08-21-08
1,f16578218940730512316d0c0,564.704552,151.617500,0.245167,8.2,0.964998,0.0,0.831785,433.0,2018-08-19-08-21-08
2,f17333786135254116214d0c0,640.170750,163.932972,0.335837,8.3,0.325308,0.0,0.831785,433.0,2018-08-19-08-21-08
3,f17914062438408532176d0c0,712.797358,189.150818,0.458709,8.4,0.994973,0.0,0.831785,433.0,2018-08-19-08-21-08
4,f12965964851209682910d0c0,788.526833,201.583246,0.462691,8.5,0.980744,0.0,0.831785,433.0,2018-08-19-08-21-08


In [15]:
len(tracks)

8811

In [ ]:
# label the tracks df by merging with test data

In [16]:
def gather_tracks(tracks):
    """
    transform the tracks df to : bee_id, [list of positions (x,y)], [list of timestamps], timestamp_of video
    """

    #  transform tracks df to : bee_id, [list of positions (x,y)], [list of timestamps], timestamp_of video
    grouped = tracks.groupby(['bee_id','track_id','video'],  as_index=False)['xpos','ypos','timestamp','zrotation']

    tracks_ml = grouped.aggregate(lambda x: list(x))

    # we don't need track_id anymore
    tracks_ml = tracks_ml.drop('track_id', 1)

    # add a column: convert video name to timestamp
    tracks_ml['video_start_time'] = tracks_ml['video'].apply(lambda x: string_to_timestamp(x))
    
    # we don't need video anymore
    tracks_ml = tracks_ml.drop('video', 1)

    # because aggregated: now multiple timestamps per row: --> rename
    tracks_ml = tracks_ml.rename(columns={'timestamp': 'timestamps'})

    # calculate start time of track by adding timestamp of track (seconds since start of video)
    # to timestamp of video (date)
    tracks_ml['track_start_time'] = tracks_ml['video_start_time'] + tracks_ml['timestamps'].apply(lambda x: x[0])
    tracks_ml['track_end_time'] = tracks_ml['video_start_time'] + tracks_ml['timestamps'].apply(lambda x: x[-1])
    
    # convert back to string
    tracks_ml['video_start_time'] = tracks_ml['video_start_time'].apply(lambda x: timestamp_to_string(x))
    
    # get end video
    tracks_ml['video_end_time'] =  tracks_ml['track_end_time'].apply(lambda x: timestamp_to_string(x))
    tracks_ml['video_end_time'] =  tracks_ml[['video_start_time','video_end_time']].apply(lambda x: get_videos_between(x[0],x[1])[-1], axis=1)
        
    return tracks_ml

def merge_tracks(tracks_ml, verbose = False):
    """
    in: 
        tracks_ml: output from gather_tracks(tracks)
    out:
        same df as in, with merged rows
        
    merge tracks of same bee where start and end timestamps are close together
    assume there can not be overlapping tracks
    """
    
    
    # 1. sort: bee_id, start_time
    tracks_ml = tracks_ml.sort_values(['bee_id', 'track_start_time'])

    
    # first convert to timestamp
    tracks_ml['video_start_time'] = tracks_ml['video_start_time'].apply(lambda x: string_to_timestamp(x))
    
    index = 0
    while(True):
        row = tracks_ml.iloc[index]
        next_row = tracks_ml.iloc[index+1]

        # if the tracks are from same bee and the start time of next row is closer then 10s -> merge the rows
        # merge rows means, next_row is deleted
        if (row['bee_id'] == next_row['bee_id']) and ((next_row['track_start_time'] - row['track_end_time']) < 10):
            
            if verbose:
                print(timestamp_to_string(row['video_start_time']), row['bee_id'], next_row['track_start_time'] - row['track_end_time'])
                print(next_row['video_start_time'] - row['video_start_time'])
                print(timestamp_to_string(row['track_start_time']),timestamp_to_string(row['track_end_time']),timestamp_to_string(row['video_start_time']),row['video_end_time'])
                print(row['timestamps'])
                print(row.name)
                print(timestamp_to_string(next_row['track_start_time']))
                print('----------------------------------')
           
            # update the timestamps of nextrow
            t = next_row['video_start_time'] - row['video_start_time']
            timestamps = list( np.array(next_row['timestamps']) + t)

            # merge xpos, ypos, timestamps lists
            tracks_ml.at[row.name,'xpos'] = row['xpos']+next_row['xpos']
            tracks_ml.at[row.name,'ypos'] = row['ypos']+next_row['ypos']
            tracks_ml.at[row.name,'timestamps'] = row['timestamps']+timestamps

            # update end_time
            tracks_ml.at[row.name,'track_end_time'] = next_row['track_end_time']
            tracks_ml.at[row.name,'video_end_time'] = next_row['video_end_time']

            # delete the merged row (next_row)
            tracks_ml = tracks_ml.drop(next_row.name).copy()

        else:
            index += 1
        
        if index == len(tracks_ml) - 1:
            break
            
    # convert back to string
    tracks_ml['video_start_time'] = tracks_ml['video_start_time'].apply(lambda x: timestamp_to_string(x))
            
    return tracks_ml

In [8]:
tracks_ml = gather_tracks(tracks)
tracks_ml = tracks_ml.sort_values(['video_start_time'])
tracks_ml.head(n=20)

NameError: name 'gather_tracks' is not defined

In [94]:
tracks_ml = merge_tracks(tracks_ml, verbose=True)

2018-08-19-13-05-40 303.0 0.09999990463256836
21.0
2018-08-19-13-05-54 2018-08-19-13-06-00 2018-08-19-13-05-40 2018-08-19-13-05-40
[14.4, 14.5, 14.7, 14.8, 14.9, 15.0, 15.1, 15.2, 15.3, 15.4, 15.5, 15.6, 15.7, 15.8, 15.9, 16.0, 16.1, 16.2, 16.3, 16.4, 16.5, 16.6, 16.7, 16.8, 16.9, 17.0, 17.1, 17.3, 17.4, 17.7, 17.8, 17.9, 18.0, 18.3, 18.6, 18.7, 18.8, 19.2, 19.3, 19.4, 19.5, 19.6, 19.7, 19.8, 19.9, 20.0, 20.1, 20.2, 20.3, 20.4, 20.5, 20.6, 20.7, 20.8, 20.9]
14
2018-08-19-13-06-01
----------------------------------
2018-08-19-13-05-40 303.0 7.400000095367432
21.0
2018-08-19-13-05-54 2018-08-19-13-06-04 2018-08-19-13-05-40 2018-08-19-13-06-01
[14.4, 14.5, 14.7, 14.8, 14.9, 15.0, 15.1, 15.2, 15.3, 15.4, 15.5, 15.6, 15.7, 15.8, 15.9, 16.0, 16.1, 16.2, 16.3, 16.4, 16.5, 16.6, 16.7, 16.8, 16.9, 17.0, 17.1, 17.3, 17.4, 17.7, 17.8, 17.9, 18.0, 18.3, 18.6, 18.7, 18.8, 19.2, 19.3, 19.4, 19.5, 19.6, 19.7, 19.8, 19.9, 20.0, 20.1, 20.2, 20.3, 20.4, 20.5, 20.6, 20.7, 20.8, 20.9, 21.0, 21.1, 21.2, 21

[5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6.0, 6.1, 6.2, 6.3, 6.4, 6.5, 6.6, 6.7, 6.8, 6.9, 7.0, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7, 7.8, 7.9, 8.0, 8.6, 8.7, 8.9, 9.2, 9.3, 9.5, 9.7, 9.8, 9.9, 10.0, 10.1, 10.7, 11.0, 11.1, 11.2, 11.3, 11.4, 11.5, 11.6, 11.7, 12.0, 12.2, 12.5, 12.6, 12.7, 12.8, 15.0, 15.1, 15.2, 15.3, 17.5, 17.7, 17.8, 18.0, 18.1, 18.2, 18.3, 18.4, 18.5, 18.6, 18.7, 20.6, 20.7, 20.8, 20.9]
130
2018-08-19-17-00-51
----------------------------------
2018-08-19-17-00-30 1301.0 0.09999990463256836
42.0
2018-08-19-17-00-35 2018-08-19-17-01-11 2018-08-19-17-00-30 2018-08-19-17-00-51
[5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6.0, 6.1, 6.2, 6.3, 6.4, 6.5, 6.6, 6.7, 6.8, 6.9, 7.0, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7, 7.8, 7.9, 8.0, 8.6, 8.7, 8.9, 9.2, 9.3, 9.5, 9.7, 9.8, 9.9, 10.0, 10.1, 10.7, 11.0, 11.1, 11.2, 11.3, 11.4, 11.5, 11.6, 11.7, 12.0, 12.2, 12.5, 12.6, 12.7, 12.8, 15.0, 15.1, 15.2, 15.3, 17.5, 17.7, 17.8, 18.0, 18.1, 18.2, 18.3, 18.4, 18.5, 18.6, 18.7, 20.6, 20.7, 20.8, 20.9, 21.0, 21.1, 21.2, 21.3, 2

In [7]:
len(tracks_ml)

NameError: name 'tracks_ml' is not defined

In [84]:
# get all tracks where bee_id and videostart_time or bee_id and video_endtime is the same

# first merge only by video_start time for in direction,
t = pd.merge(tracks_ml,test_data[['bee_id','in_direction','video_start_time']], on=['bee_id','video_start_time'], how='inner')

In [85]:
len(t)

70

In [86]:
t2 = pd.merge(tracks_ml,test_data[['bee_id','out_direction','video_end_time']], on=['bee_id','video_end_time'], how='inner')
len(t2)

69

In [24]:
# then merge by video end time for out direction
t2 = pd.merge(t,test_data[['bee_id','out_direction','video_end_time']], on=['bee_id','video_end_time'], how='inner')
len(t2)

64

In [97]:
69/172

0.4011627906976744

# Get labels from test_set

In [67]:
# read in the test data csv
test_data = pd.read_csv('bees_test.csv')

# convert the full filenames to string timestamps, and sort by timestamp_in
test_data['video'] = test_data['video'].apply(lambda x: filename_to_datestring(x))
test_data['video_start_time'] = test_data['timestamp_in'].apply(lambda x: filename_to_datestring(x))
test_data['video_end_time'] = test_data['timestamp_out'].apply(lambda x: filename_to_datestring(x))
test_data.sort_values(['video_start_time'], inplace=True)

test_data.drop('timestamp_in', 1, inplace=True)
test_data.drop('timestamp_out', 1, inplace=True)
test_data.drop('video', 1, inplace=True)

# get all tracks where bee_id, videio_start_time and video_end_time are the same
tracks_ml = pd.merge(tracks_ml,test_data,how='inner',on=['bee_id','video_start_time','video_end_time'])

# we only look at tracks where  in_direction and out_direction are different
tracks_ml = tracks_ml[(tracks_ml['in_direction'] != tracks_ml['out_direction'])]

# we assing movement based on in_ and out_directoin
tracks_ml.loc[(tracks_ml['in_direction'] == 'left') & (tracks_ml['out_direction'] == 'right'),'movement'] = 1
tracks_ml.loc[(tracks_ml['in_direction'] == 'right') & (tracks_ml['out_direction'] == 'left'),'movement'] = 0

tracks_ml.drop('in_direction', 1, inplace=True)
tracks_ml.drop('out_direction', 1, inplace=True)

tracks_ml = tracks_ml['bee_id', 'xpos', 'ypos', 'zrotation', 'timestamps', 'video_start_time', 'video_end_time', 'track_start_time', 'track_end_time', 'video',
       'movement']

In [70]:
tracks_ml.columns

Index(['bee_id', 'xpos', 'ypos', 'timestamps', 'zrotation', 'video_start_time',
       'track_start_time', 'track_end_time', 'video_end_time', 'video',
       'movement'],
      dtype='object')

In [69]:
tracks_ml.head()

,bee_id,xpos,ypos,timestamps,zrotation,video_start_time,track_start_time,track_end_time,video_end_time,video,movement
0,220.0,"[587.6349468231201, 577.2753982543945, 589.496...","[123.45260810852051, 125.9588794708252, 138.67...","[0.4, 14.0, 14.1, 14.2, 14.3, 14.4, 14.5, 14.6...","[-2.787043, 0.020939333, -0.15247042, 0.053603...",2018-08-19-05-25-51,1.534649e+09,1.534649e+09,2018-08-19-05-25-51,2018-08-19-05-25-51,1.0
4,388.0,"[1375.2675552368164, 1313.4858207702634, 1225....","[239.0948257446289, 238.7294406890869, 237.849...","[0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1]","[2.9390156000000003, 2.9522842999999996, -2.97...",2018-08-19-08-29-06,1.534660e+09,1.534660e+09,2018-08-19-08-29-06,2018-08-19-08-29-06,0.0
5,420.0,"[1075.0250930786133, 637.5782222747803, 538.01...","[187.6749038696289, 211.6030120849609, 210.762...","[13.6, 14.0, 14.1, 14.2]","[2.9731865, 2.9255147, 2.7518196, 2.5865266]",2018-08-19-09-48-00,1.534665e+09,1.534665e+09,2018-08-19-09-48-00,2018-08-19-09-48-00,0.0
6,446.0,"[477.1684074401856, 526.4452629089354, 601.512...","[151.28521347045896, 152.16980743408203, 138.8...","[20.6, 20.7, 20.9, 21.0, 21.1, 21.2, 21.3, 21....","[0.029027572, 0.03262988, 0.072178215]",2018-08-19-09-47-39,1.534665e+09,1.534665e+09,2018-08-19-09-48-00,2018-08-19-09-48-00,1.0
7,475.0,"[526.7946376800537, 563.8891906738281, 638.246...","[139.16316413879395, 139.3912754058838, 139.43...","[4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5.0, ...","[0.056613013, 0.28731626, 0.3744915, 0.1337887...",2018-08-19-14-45-46,1.534683e+09,1.534683e+09,2018-08-19-14-45-46,2018-08-19-14-45-46,1.0


In [59]:

x = "left"
y = "left"


x[1] = 0
print(y)

TypeError: 'str' object does not support item assignment

In [48]:
len(tracks_ml)

62

In [ ]:
# create a featur vector for ML
# columns for output: bee_id, time_in, time_out, os_in, os_out, in_direction, out_direction
# feature_columns: timestamp_in, duration, in_z_rotation, out_z_rotation, in_coordinate, out_coordinate, n_detections

# get time_in and time_out
t = tracks_ml.iloc[0]

In [ ]:
tracks_ml['video_start'] = tracks_ml['video_start_time'].apply(lambda x: timestamp_to_string(round(x)))
tracks_ml['start_time'] = tracks_ml['start_time'].apply(lambda x: timestamp_to_string(round(x)))
tracks_ml['end_time'] = tracks_ml['end_time'].apply(lambda x: timestamp_to_string(round(x)))

In [ ]:
t['video_start_time'], t['start_time']

In [ ]:
get_videos_between(t['video_start_time'], t['end_time'])

there is a problem in bb_behaviour: the timestamps always start at 00, this means it is possible that timestamps are not monotonic ascending

TODO: first clamp timestamps: so that last timestamps are floored
```   
timestamps = timestamps + time_start_video
timestamps = clamp(timestamps,0,floor(timestamps))
timestamps = timestamps - time_start_video
```
then some timestamps are double, but this is not a problem?

In [ ]:
len(all_videos)

In [73]:
import random

In [78]:
random.randint(0,1)

0